In [ ]:
import numpy, whisper, torch
from pathlib import Path
import jiwer
from jiwer import *
import csv
import unicodedata
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import math
import wave
import re
import torch




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/Tiempo_de_Vida/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/Tiempo_de_Vida/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/Tiempo_de_Vida/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/Tiempo_de_Vida/lib/python3.10/site-packages/traitlets/config/application.py", line 10

## Normalizaciones

Como explicamos, evaluaremos las transcripciones "Crudas" (sin ninguna transformación) y "Normalizadas" (Sin puntuaciones ni mayusculas)

#### Transformaciones para transcripciones normalizadas

In [2]:
#Transformaciones para Palabras
tr_w = jiwer.Compose([
    Strip(), #quitamos espacios al comienzo y al final
    ToLowerCase(), #pasamos todo a minúsculas   
    RemovePunctuation(), #borramos comas, puntos y signos              
    RemoveWhiteSpace(replace_by_space=True), #limpiamos espacios extra
    RemoveMultipleSpaces(), #idem
    ReduceToListOfListOfWords(), #convertimos las frases en listas de listas de palabras (para contar errores por palabra)
])

#Transformaciones para Caracteres
tr_c = jiwer.Compose([
    Strip(),
    ToLowerCase(),
    RemovePunctuation(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfChars(), #lo llevamos a listas de caracteres en lugar de palabras
])


#### Transformaciones para transcripciones crudas:
Se aplican solo durante el cálculo del WER y CER de los fragmentos de los audios. Aquí se eliminan comas y signos de pregunta debido a que las referencias de estos fragmentos no contaban con las primeras y además los signos de interrogación eran usados además para marcar palabras inentendibles al momento de la escucha.

In [3]:
#Para hipotesis de los audios fragmentados - Palabras - 
tr_w_crudo_sin_preguntas = Compose([
    Strip(),
    ToLowerCase(),
    SubstituteRegexes({
        r",": " ",       # elimina las comas
        r"\?": " ",      # elimina los signos de pregunta
    }),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(),
])

#Para hipotesis de los audios fragmentados - Caracteres - 
tr_c_crudo_sin_preguntas = Compose([
    Strip(),
    ToLowerCase(),
    SubstituteRegexes({
        r",": " ",       # elimina las comas
        r"\?": " ",      # elimina los signos de pregunta
    }),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfChars(),
])


#Para referencias de los audios fragmentados - Palabras - 
tr_w_crudoREF_sin_preguntas = Compose([
    SubstituteRegexes({
        r"\?": " ",      # elimina los signos de pregunta
    }),
    ReduceToListOfListOfWords(),
])

#Para referencias de los audios fragmentados - Palabras - 
tr_c_crudoREF_sin_preguntas = Compose([
    SubstituteRegexes({
        r"\?": " ",      # elimina los signos de pregunta
    }),
    ReduceToListOfListOfChars(),
])

Estas transformaciones restantes son utilizadas para visualizar de mejor manera los resultados al momento de imprimir las transcripciones. Son identicas a las anteriores salvo que no convierten los textos a listas de palabras o caracteres.

In [ ]:
#Normalizado:
tr= jiwer.Compose([
    Strip(),
    ToLowerCase(),          
    RemovePunctuation(),               
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
])

#Crudo:
#PARA HIPÓTESIS:
tr_crudo_sin_preguntas = Compose([
    Strip(),
    ToLowerCase(),
    SubstituteRegexes({
        r",": " ",
        r"\?": " ",      # elimina los signos de pregunta
    }),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
])

#PARA REFERENCIA:
tr_crudoREF_sin_preguntas = Compose([
    SubstituteRegexes({
        r"\?": " ",      # elimina los signos de pregunta
    }),
])


#### Cálculo de métricas:

Dada una referencia (transcripción a mano del audio) y una hipótesis (output transcripto por el modelo), devuelve un diccionario de métricas asociadas a ese audio y transcripción. 
Las funciones son diferentes si se trata de transcripciones crudas, normalizadas y además si son fragmentos. Esto se debe a la necesidad de utilizar diferentes transformaciones

In [ ]:
def calcular_metricas(refs:str, hyps: str):
    metricas = dict()

    # Word Error Rate
    out_wer = jiwer.process_words(refs, hyps, reference_transform=tr_w, hypothesis_transform=tr_w)
    # Character Error Rate
    out_cer = jiwer.process_characters(refs, hyps, reference_transform=tr_c, hypothesis_transform=tr_c)

    metricas["WER"] = out_wer.wer
    metricas["CER"] = out_cer.cer
    metricas["INFO_ALINEACION"] = jiwer.visualize_alignment(out_wer) #genera una representación del texto de cómo se alinean las palabras del REF con el HYP y se agregan marcas S, I, D. Además, un resumen con número de oraciones, s,i,d, y métricas derivadas: mer, wil, wip, wer
    metricas["INFO_ERRORES"] = jiwer.visualize_error_counts(out_wer) #genera un desglose de los errores concretos palabra por palabra (te indica todas las sustituciones, inserciones y borrados)
    metricas["TIPOS DE ERRORES"] = dict()
    metricas["TIPOS DE ERRORES"]["SUSTITUCIONES"] = out_wer.substitutions
    metricas["TIPOS DE ERRORES"]["INSERCIONES"] = out_wer.insertions
    metricas["TIPOS DE ERRORES"]["BORRADOS"] = out_wer.deletions
  
    return metricas



In [ ]:
def calcular_metricas_crudo(refs:str, hyps: str):
    #DADA UNA REFERENCIA (transcripción real) Y UNA HIPÓTESIS (output transcripto por el modelo) para un cierto MODELO, devuelve un diccionario de métricas asociadas a ese audio y transcripción
    #CONSERVAMOS SIGNOS DE PREGUNTA (PARA LOS ORIGINALES)
    metricas = dict()

    # Word Error Rate
    out_wer = jiwer.process_words(refs, hyps)
    # Character Error Rate
    out_cer = jiwer.process_characters(refs, hyps)

    metricas["WER"] = out_wer.wer
    metricas["CER"] = out_cer.cer
    metricas["INFO_ALINEACION"] = jiwer.visualize_alignment(out_wer) #genera una representación del texto de cómo se alinean las palabras del REF con el HYP y se agregan marcas S, I, D. Además, un resumen con número de oraciones, s,i,d, y métricas derivadas: mer, wil, wip, wer
    metricas["INFO_ERRORES"] = jiwer.visualize_error_counts(out_wer) #genera un desglose de los errores concretos palabra por palabra (te indica todas las sustituciones, inserciones y borrados)
    metricas["TIPOS DE ERRORES"] = dict()
    metricas["TIPOS DE ERRORES"]["SUSTITUCIONES"] = out_wer.substitutions
    metricas["TIPOS DE ERRORES"]["INSERCIONES"] = out_wer.insertions
    metricas["TIPOS DE ERRORES"]["BORRADOS"] = out_wer.deletions
  
    return metricas




def calcular_metricas_crudo_FRAGMENTOS(refs:str, hyps: str):
    #DADA UNA REFERENCIA (transcripción real) Y UNA HIPÓTESIS (output transcripto por el modelo) para un cierto MODELO, devuelve un diccionario de métricas asociadas a ese audio y transcripción
    #QUITAMOS SIGNOS DE PREGUNTA (PARA LOS FRAGMENTOS)
    metricas = dict()

    # Word Error Rate
    out_wer = jiwer.process_words(refs, hyps, reference_transform=tr_w_crudoREF_sin_preguntas, hypothesis_transform=tr_w_crudo_sin_preguntas)
    # Character Error Rate
    out_cer = jiwer.process_characters(refs, hyps, reference_transform=tr_c_crudoREF_sin_preguntas, hypothesis_transform=tr_c_crudo_sin_preguntas)

    metricas["WER"] = out_wer.wer
    metricas["CER"] = out_cer.cer
    metricas["INFO_ALINEACION"] = jiwer.visualize_alignment(out_wer) #genera una representación del texto de cómo se alinean las palabras del REF con el HYP y se agregan marcas S, I, D. Además, un resumen con número de oraciones, s,i,d, y métricas derivadas: mer, wil, wip, wer
    metricas["INFO_ERRORES"] = jiwer.visualize_error_counts(out_wer) #genera un desglose de los errores concretos palabra por palabra (te indica todas las sustituciones, inserciones y borrados)
    metricas["TIPOS DE ERRORES"] = dict()
    metricas["TIPOS DE ERRORES"]["SUSTITUCIONES"] = out_wer.substitutions
    metricas["TIPOS DE ERRORES"]["INSERCIONES"] = out_wer.insertions
    metricas["TIPOS DE ERRORES"]["BORRADOS"] = out_wer.deletions
  
    return metricas

#### Promedio de las métricas:
Se calcula el promedio de las metricas para cada tarea diferente. Se basa en un diccionario que contiene todas las transcripciones para las 3 tareas. Será generado más adelante en el archivo

In [ ]:
def metricas_promedio(models_data:dict[dict[str:str]]):
    promediosWER = dict()
    promediosCER = dict()
    
    sumaWER_TAREA1 = 0
    sumaCER_TAREA1 = 0
    sumaWER_TAREA2 = 0
    sumaCER_TAREA2 = 0
    sumaWER_TAREA3 = 0
    sumaCER_TAREA3 = 0

    count_tarea1 = 0
    count_tarea2 = 0
    count_tarea3 = 0

    for registro in models_data.keys(): #Recorrer por audios
            for clave in models_data[registro].keys(): #3 ITERACIONES: TAREAS 1, 2 Y 3
                  for fragmento in models_data[registro][clave].keys(): #Si la tarea es la 1 -> una única iteración (el audio completo), si la tarea es la 3, recorre todos los fragmentos del registro correspondiente
                        if clave == "TAREA 1": 
                              sumaWER_TAREA1 += models_data[registro][clave][fragmento]["METRICS"]["WER"]
                              sumaCER_TAREA1 += models_data[registro][clave][fragmento]["METRICS"]["CER"]
                              count_tarea1 += 1
                        elif clave == "TAREA 2":
                              sumaWER_TAREA2 += models_data[registro][clave][fragmento]["METRICS"]["WER"]
                              sumaCER_TAREA2 += models_data[registro][clave][fragmento]["METRICS"]["CER"]
                              count_tarea2 += 1
                        elif clave == "TAREA 3":
                              sumaWER_TAREA3 += models_data[registro][clave][fragmento]["METRICS"]["WER"]
                              sumaCER_TAREA3 += models_data[registro][clave][fragmento]["METRICS"]["CER"]
                              count_tarea3 += 1

    promedioWER_tarea1 = sumaWER_TAREA1/count_tarea1
    promedioCER_tarea1 = sumaCER_TAREA1/count_tarea1
    promedioWER_tarea2 = sumaWER_TAREA2/count_tarea2
    promedioCER_tarea2 = sumaCER_TAREA2/count_tarea2
    promedioWER_tarea3 = sumaWER_TAREA3/count_tarea3
    promedioCER_tarea3 = sumaCER_TAREA3/count_tarea3   

    promediosWER["TAREA 1"] = promedioWER_tarea1
    promediosWER["TAREA 2"] = promedioWER_tarea2
    promediosWER["TAREA 3"] = promedioWER_tarea3

    promediosCER["TAREA 1"] = promedioCER_tarea1
    promediosCER["TAREA 2"] = promedioCER_tarea2
    promediosCER["TAREA 3"] = promedioCER_tarea3
        
    return promediosWER, promediosCER



def metricas_promedio_crudo(models_data:dict[dict[str:str]]):
    promediosWER = dict()
    promediosCER = dict()
    
    sumaWER_TAREA1 = 0
    sumaCER_TAREA1 = 0
    sumaWER_TAREA2 = 0
    sumaCER_TAREA2 = 0
    sumaWER_TAREA3 = 0
    sumaCER_TAREA3 = 0

    count_tarea1 = 0
    count_tarea2 = 0
    count_tarea3 = 0

    for registro in models_data.keys(): #Recorrer por audios
            for clave in models_data[registro].keys(): #3 ITERACIONES: TAREAS 1, 2 Y 3
                  for fragmento in models_data[registro][clave].keys(): #Si la tarea es la 1 -> una única iteración (el audio completo), si la tarea es la 3, recorre todos los fragmentos del registro correspondiente
                        if clave == "TAREA 1": 
                              sumaWER_TAREA1 += models_data[registro][clave][fragmento]["RAW METRICS"]["WER"]
                              sumaCER_TAREA1 += models_data[registro][clave][fragmento]["RAW METRICS"]["CER"]
                              count_tarea1 += 1
                        elif clave == "TAREA 2":
                              sumaWER_TAREA2 += models_data[registro][clave][fragmento]["RAW METRICS"]["WER"]
                              sumaCER_TAREA2 += models_data[registro][clave][fragmento]["RAW METRICS"]["CER"]
                              count_tarea2 += 1
                        elif clave == "TAREA 3":
                              sumaWER_TAREA3 += models_data[registro][clave][fragmento]["RAW METRICS"]["WER"]
                              sumaCER_TAREA3 += models_data[registro][clave][fragmento]["RAW METRICS"]["CER"]
                              count_tarea3 += 1

    promedioWER_tarea1 = sumaWER_TAREA1/count_tarea1
    promedioCER_tarea1 = sumaCER_TAREA1/count_tarea1
    promedioWER_tarea2 = sumaWER_TAREA2/count_tarea2
    promedioCER_tarea2 = sumaCER_TAREA2/count_tarea2
    promedioWER_tarea3 = sumaWER_TAREA3/count_tarea3
    promedioCER_tarea3 = sumaCER_TAREA3/count_tarea3   

    promediosWER["TAREA 1"] = promedioWER_tarea1
    promediosWER["TAREA 2"] = promedioWER_tarea2
    promediosWER["TAREA 3"] = promedioWER_tarea3

    promediosCER["TAREA 1"] = promedioCER_tarea1
    promediosCER["TAREA 2"] = promedioCER_tarea2
    promediosCER["TAREA 3"] = promedioCER_tarea3
        
    return promediosWER, promediosCER

#### Transcripción y confección del diccionario principal que almacenará todos nuestros resultados:
A partir de un csv con la ruta a los audios y sus referencias, se realizan las correspondientes transcripciones y se almacenan en el diccionario "results" junto a los WER Y CER para las versiones crudas y normalizadas. 

El diccionario result se organizará de la siguiente manera:

results[nombre del audio global][tarea 1, 2 o 3][nombre del archivo derivado][referencia, hipotesis o métricas]

La diferencia entre el nombre del audio global y el nombre del archivo derivado es que el primero se comparte entre las tres tareas y el último es unico para cada fragmento, audio reconstruido y audio cpmpleto.

In [ ]:
def eval(data_archivo:str, modelo:str, models_data, audio_root: str, clave: str):
    model = whisper.load_model(modelo)

    carpeta_audios = Path(audio_root)
    print("LA CARPETA ES: ", carpeta_audios)
    if not carpeta_audios.exists():
        raise FileNotFoundError(f"Carpeta de audios no encontrada: {audio_root}")

    with open(data_archivo, encoding="utf-8") as f:
        r = csv.DictReader(f, delimiter=",")
        
        #RECORREMOS CADA REGISTRO DEL CSV
        for registro in r: 
            #GUARDAMOS EL AUDIO WAV Y SU REFERENCIA 
            audio = registro['AUDIO']
            ref = registro["TRANSCRIPCION"]
            origen = registro["AUDIO_ORIGINAL"]
            audio_path = carpeta_audios / audio
            print("EL PATH ES: ", audio_path)
            #TRANSCRIBIMOS, OBTENIENDO LA HIPÓTESIS DEL MODELO
            output = model.transcribe(str(audio_path), language="es", fp16=False, temperature=0.0)
            hyp = output["text"]


            #CALCULAMOS MÉTRICAS
            metricas = calcular_metricas(ref, hyp)
            if clave == "TAREA 1" or clave == "TAREA 2":
                metricas_crudo = calcular_metricas_crudo(ref, hyp)
            elif clave == "TAREA 3":
                metricas_crudo = calcular_metricas_crudo_FRAGMENTOS(ref, hyp)
            
            audio_key = origen
            print(audio_key)
            if audio_key not in models_data.keys():
                models_data[audio_key] = dict()
            if clave not in models_data[audio_key].keys():
                models_data[audio_key][clave] = dict()

            audio_tarea = Path(audio).stem 
            models_data[audio_key][clave][audio_tarea] = dict()
            
            if clave == "TAREA 1" or clave == "TAREA 2":
                models_data[audio_key][clave][audio_tarea]["REF CRUDA"] = ref
                models_data[audio_key][clave][audio_tarea]["HYP CRUDA"] = hyp
            elif clave == "TAREA 3": 
                models_data[audio_key][clave][audio_tarea]["REF CRUDA"] = tr_crudoREF_sin_preguntas(ref) #solo sacamos signos de pregunta
                models_data[audio_key][clave][audio_tarea]["HYP CRUDA"] = tr_crudo_sin_preguntas(hyp) #sacamos signos de pregunta + trasformaciones que quitan comas y mayúsculas


            models_data[audio_key][clave][audio_tarea]["REF"] = tr(ref)
            models_data[audio_key][clave][audio_tarea]["HYP"] = tr(hyp)
            models_data[audio_key][clave][audio_tarea]["OUTPUT TOTAL"] = output
            models_data[audio_key][clave][audio_tarea]["METRICS"] = dict()
            models_data[audio_key][clave][audio_tarea]["METRICS"]["WER"] = metricas["WER"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["CER"] = metricas["CER"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["INFO_ALINEACION"] = metricas["INFO_ALINEACION"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["INFO_ERRORES"] = metricas["INFO_ERRORES"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["TIPOS DE ERRORES"] = dict()
            models_data[audio_key][clave][audio_tarea]["METRICS"]["TIPOS DE ERRORES"]["SUSTITUCIONES"] = metricas["TIPOS DE ERRORES"]["SUSTITUCIONES"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["TIPOS DE ERRORES"]["INSERCIONES"] = metricas["TIPOS DE ERRORES"]["INSERCIONES"]
            models_data[audio_key][clave][audio_tarea]["METRICS"]["TIPOS DE ERRORES"]["BORRADOS"] = metricas["TIPOS DE ERRORES"]["BORRADOS"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"] = dict()
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["WER"] = metricas_crudo["WER"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["CER"] = metricas_crudo["CER"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["INFO_ALINEACION"] = metricas_crudo["INFO_ALINEACION"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["INFO_ERRORES"] = metricas_crudo["INFO_ERRORES"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["TIPOS DE ERRORES"] = dict()
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["TIPOS DE ERRORES"]["SUSTITUCIONES"] = metricas_crudo["TIPOS DE ERRORES"]["SUSTITUCIONES"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["TIPOS DE ERRORES"]["INSERCIONES"] = metricas_crudo["TIPOS DE ERRORES"]["INSERCIONES"]
            models_data[audio_key][clave][audio_tarea]["RAW METRICS"]["TIPOS DE ERRORES"]["BORRADOS"] = metricas_crudo["TIPOS DE ERRORES"]["BORRADOS"]

                


#### Imprimir de una forma amigable el diccionario results

In [ ]:
def printear_models_data(models_data: dict):
    print("\n================= RESULTADOS =================\n")
    
    for audio_key, tareas in models_data.items():
        print(f"AUDIO ORIGINAL: {audio_key}\n")
        
        for clave_tarea, fragmentos in tareas.items():
            print(f"  >> {clave_tarea}")
            
            for audio_tarea, info in fragmentos.items():
                print(f"\n    FRAGMENTO: {audio_tarea}")

                print(f"        OUTPUT TOTAL: {info['OUTPUT TOTAL']}\n")


                print(f"        REF: {info['REF']}")
                print(f"        HYP: {info['HYP']}\n")
                
                # --- Métricas procesadas ---
                m = info["METRICS"]
                print("        MÉTRICAS PROCESADAS:")
                print(f"            - WER: {m['WER']:.3f}")
                print(f"            - CER: {m['CER']:.3f}")
                print("            - ALINEACIÓN (palabras):")
                print(f"                  {m['INFO_ALINEACION']}")
                print("            - ERRORES (resumen):")
                print(f"                  {m['INFO_ERRORES']}")
                print("            - TIPOS DE ERRORES:")
                print(f"                  Sustituciones: {m['TIPOS DE ERRORES']['SUSTITUCIONES']}")
                print(f"                  Inserciones:   {m['TIPOS DE ERRORES']['INSERCIONES']}")
                print(f"                  Borrados:      {m['TIPOS DE ERRORES']['BORRADOS']}")
                print()
                
                
                print(f"        REF cruda: {info['REF CRUDA']}")
                print(f"        HYP cruda: {info['HYP CRUDA']}\n")
                # --- Métricas crudas ---
                n = info["RAW METRICS"]
                print("        MÉTRICAS CRUDAS:")
                print(f"            - WER: {n['WER']:.3f}")
                print(f"            - CER: {n['CER']:.3f}")
                print("            - ALINEACIÓN (palabras):")
                print(f"                  {n['INFO_ALINEACION']}")
                print("            - ERRORES (resumen):")
                print(f"                  {n['INFO_ERRORES']}")
                print("            - TIPOS DE ERRORES:")
                print(f"                  Sustituciones: {n['TIPOS DE ERRORES']['SUSTITUCIONES']}")
                print(f"                  Inserciones:   {n['TIPOS DE ERRORES']['INSERCIONES']}")
                print(f"                  Borrados:      {n['TIPOS DE ERRORES']['BORRADOS']}")
                print("\n--------------------------------------------------\n")
        print("==================================================\n")
    
    print("================= FIN RESULTADOS =================\n")

#### Código de evaluación principa

Tomando el modelo Turbo, el cual es alterable, ejecutamos las 3 tareas para el total de los audios 

In [ ]:
modelos = ["turbo"]
niños_data = dict()


for modelo in modelos: 
    #Tarea 1: TRANSCRIPCIÓN DEL AUDIO ORIGINAL 
    archivo = "docs_niños/completos.csv"
    audio_root = "audios_niños/Audios completos"
    clave = "TAREA 1"
    eval(archivo, modelo, niños_data, audio_root, clave)

    #Tarea 2: TRANSCRIPCIÓN DEL AUDIO CONCATENADO
    archivo = "docs_niños/reconstruidos.csv"
    audio_root = "audios_niños/Audios reconstruidos"
    clave = "TAREA 2"
    eval(archivo, modelo, niños_data, audio_root, clave) 


    #Tarea 3: TRANSCRIPCIÓN DE FRAGMENTOS 
    archivo = "docs_niños/fragmentos.csv"
    audio_root = "audios_niños/Audios fragmentados"
    clave = "TAREA 3"
    eval(archivo, modelo, niños_data, audio_root, clave)
    print("TAREA 3 FINALIZADA")

In [ ]:
printear_models_data(niños_data)

Con los resultados en nuestras manos, calculamos el WER y el CER promedio para cada tarea tanto para las versiones crudas como normalizadas

In [ ]:
WERs_promedios, CERs_promedios = metricas_promedio(niños_data)
print("\nWER PROMEDIO DE TAREA 1:")
print(WERs_promedios["TAREA 1"])
print("\nCER PROMEDIO DE TAREA 1:")
print(CERs_promedios["TAREA 1"])

print("\nWER PROMEDIO DE TAREA 2:")
print(WERs_promedios["TAREA 2"])
print("\nCER PROMEDIO DE TAREA 2:")
print(CERs_promedios["TAREA 2"])

print("\nWER PROMEDIO DE TAREA 3:")
print(WERs_promedios["TAREA 3"])
print("\nCER PROMEDIO DE TAREA 3:")
print(CERs_promedios["TAREA 3"])

WERs_promedios_crudo, CERs_promedios_crudo = metricas_promedio_crudo(niños_data)
    
print("\nWER PROMEDIO DE TAREA 1 CRUDO:")
print(WERs_promedios_crudo["TAREA 1"])
print("\nCER PROMEDIO DE TAREA 1 CRUDO:")
print(CERs_promedios_crudo["TAREA 1"])

print("\nWER PROMEDIO DE TAREA 2 CRUDO:")
print(WERs_promedios_crudo["TAREA 2"])
print("\nCER PROMEDIO DE TAREA 2 CRUDO:")
print(CERs_promedios_crudo["TAREA 2"])

print("\nWER PROMEDIO DE TAREA 3 CRUDO:")
print(WERs_promedios_crudo["TAREA 3"])
print("\nCER PROMEDIO DE TAREA 3 CRUDO:")
print(CERs_promedios_crudo["TAREA 3"])


Guardar en un archivo Excel

In [ ]:
def _tarea_num(clave_tarea: str) -> int:
    m = re.search(r"(\d+)", str(clave_tarea))
    return int(m.group(1)) if m else 999

def _natural_key(s: str):
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", str(s))]

def _safe_get(d, *keys, default=""):
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

def exportar_excel_asr(models_data: dict, path="resultados.xlsx"):
    """
    Genera un Excel con filas en orden:
      AUDIO, TAREA 1, TAREA 2, TAREA 3 (un renglón por fragmento).
    Columnas:
      AUDIO, TAREA, FRAGMENTO,
      REF CRUDA, HYP CRUDA, REF, HYP,
      WER, CER, SUSTITUCIONES, INSERCIONES, BORRADOS,
      WER crudo, CER crudo, SUSTITUCIONES crudo, INSERCIONES crudo, BORRADOS crudo
    """
    filas = []

    for audio_key in sorted(models_data.keys()):
        tareas = models_data[audio_key]

        for tn in [1, 2, 3]:
            # buscar la clave 'TAREA X'
            clave_tarea = None
            for k in tareas.keys():
                if _tarea_num(k) == tn:
                    clave_tarea = k
                    break
            if clave_tarea is None:
                continue

            fragmentos = tareas[clave_tarea]
            for frag in sorted(fragmentos.keys(), key=_natural_key):
                info = fragmentos[frag]

                # Textos crudos y transformados
                ref_cruda = _safe_get(info, "REF CRUDA", default="")
                hyp_cruda = _safe_get(info, "HYP CRUDA", default="")
                ref_txt   = _safe_get(info, "REF", default="")
                hyp_txt   = _safe_get(info, "HYP", default="")

                # Métricas procesadas
                m = _safe_get(info, "METRICS", default={})
                mt = _safe_get(m, "TIPOS DE ERRORES", default={})

                # Métricas crudas
                n = _safe_get(info, "RAW METRICS", default={})
                nt = _safe_get(n, "TIPOS DE ERRORES", default={})

                filas.append({
                    "AUDIO": audio_key,
                    "TAREA": tn,
                    "FRAGMENTO": frag if tn == 3 else "",

                    "REF CRUDA": ref_cruda,
                    "HYP CRUDA": hyp_cruda,
                    "REF": ref_txt,
                    "HYP": hyp_txt,

                    # Procesadas
                    "WER": _safe_get(m, "WER", default=""),
                    "CER": _safe_get(m, "CER", default=""),
                    "SUSTITUCIONES": _safe_get(mt, "SUSTITUCIONES", default=0),
                    "INSERCIONES":  _safe_get(mt, "INSERCIONES",  default=0),
                    "BORRADOS":     _safe_get(mt, "BORRADOS",     default=0),

                    # Crudas
                    "WER crudo": _safe_get(n, "WER", default=""),
                    "CER crudo": _safe_get(n, "CER", default=""),
                    "SUSTITUCIONES crudo": _safe_get(nt, "SUSTITUCIONES", default=0),
                    "INSERCIONES crudo":  _safe_get(nt, "INSERCIONES",  default=0),
                    "BORRADOS crudo":     _safe_get(nt, "BORRADOS",     default=0),
                })

    cols = [
        "AUDIO", "TAREA", "FRAGMENTO",
        "REF CRUDA", "HYP CRUDA", "REF", "HYP",
        "WER", "CER", "SUSTITUCIONES", "INSERCIONES", "BORRADOS",
        "WER crudo", "CER crudo", "SUSTITUCIONES crudo", "INSERCIONES crudo", "BORRADOS crudo",
    ]
    df = pd.DataFrame(filas, columns=cols)

    # ---- Exportar con formato (XlsxWriter) ----
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        sheet = "ASR"
        df.to_excel(writer, index=False, sheet_name=sheet)

        wb = writer.book
        ws = writer.sheets[sheet]

        wrap_fmt   = wb.add_format({"text_wrap": True, "valign": "top"})
        normal_fmt = wb.add_format({"valign": "vcenter"})
        num_fmt4   = wb.add_format({"valign": "vcenter", "num_format": "0.0000"})

        max_width_cap = 80
        min_width_text = 12
        numeric_cols = {"WER", "CER", "WER crudo", "CER crudo"}

        for col_idx, col_name in enumerate(df.columns):
            series = df[col_name].astype(str) if not df.empty else pd.Series([col_name])
            max_len = max([len(col_name)] + [len(s) for s in series])

            if col_name in ("REF CRUDA", "HYP CRUDA", "REF", "HYP"):
                width = min(max(max_len, 40), max_width_cap)
                ws.set_column(col_idx, col_idx, width, wrap_fmt)
            elif col_name in numeric_cols:
                ws.set_column(col_idx, col_idx, 12, num_fmt4)
            else:
                width = min(max(max_len + 2, min_width_text), 28)
                ws.set_column(col_idx, col_idx, width, normal_fmt)

        # Alto de fila según REF/HYP (transformadas) para que se vean completas
        base_height = 15
        char_per_line = 80
        for row_idx in range(len(df)):
            lines_needed = 1
            for col in ("REF", "HYP"):
                text = str(df.at[row_idx, col]).strip()
                if text:
                    logical_lines = text.count("\n") + 1
                    wrapped_lines = math.ceil(len(text) / char_per_line)
                    lines_needed = max(lines_needed, logical_lines, wrapped_lines)
            ws.set_row(row_idx + 1, base_height * lines_needed, wrap_fmt)

    print(f"Archivo guardado en {path}")

In [ ]:
exportar_excel_asr(niños_data, path="resultados_experimentacion_niños.xlsx")